In [46]:
"""Cliente para la API de Analytics.

Incluye funcionalidades para descargar las métricas y dimensiones
de una vista determinada.

Para autenticarse en la API de Analytics solo basta con utilizar el json de la cuenta de servicio.

"""

# Arreglar el path
import sys
import json

import os
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import timedelta, datetime


SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

DATE_FORMAT = '%Y-%m-%d'



### Función que convierte la response de la api en un dataframe de Pandas
def ga_response_dataframe(response):
    """ Función para crear un dataframe de pandas a través de la respuesta de google Analytics
    Args:
        response: respuesta de la petición de Google Analytics
    Returns:
        Dataframe de pandas con las diferentes dimensiones y métricas
    """
    row_list = []

    # Get each collected report
    for report in response.get('reports', []):
        # Set column headers
        column_header = report.get('columnHeader', {})
        dimension_headers = column_header.get('dimensions', [])
        metric_headers = column_header.get('metricHeader', {}).get('metricHeaderEntries', [])

        # Get each row in the report
        for row in report.get('data', {}).get('rows', []):
            # create dict for each row
            row_dict = {}
            dimensions = row.get('dimensions', [])
            date_range_values = row.get('metrics', [])

            # Fill dict with dimension header (key) and dimension value (value)
            for header, dimension in zip(dimension_headers, dimensions):
                row_dict[header] = dimension

            # Fill dict with metric header (key) and metric value (value)
            for i, values in enumerate(date_range_values):
                for metric, value in zip(metric_headers, values.get('values')):
                    # Set int as int, float a float
                    if ',' in value or '.' in value:
                        row_dict[metric.get('name')] = float(value)
                    else:
                        row_dict[metric.get('name')] = int(value)

            row_list.append(row_dict)
    return pd.DataFrame(row_list)


class Client:
    """Cliente para conexión Oauth2 con la API de Taboola."""

    def __init__(self, path_cred):
        """Initializes an Analytics Reporting API V4 service object.
        """

        credentials = ServiceAccountCredentials.from_json_keyfile_name(
            path_cred, SCOPES)

        # Build the service object.
        self.analytics = build('analyticsreporting', 'v4', credentials=credentials)

    ### Función que recibe diferentes parámetros para extraer datos de Analytics
    def get_report(self,
                   view_id ,
                   start_date ,
                   end_date ,
                   list_metrics = [ 'ga:users' , 'ga:newUsers' , 'ga:sessions' , 'ga:bounceRate' ,
                                    'ga:pageviewsPerSession' , 'ga:avgSessionDuration' ] ,
                   list_dimensions = [ 'ga:source','ga:medium' ] ,
                   # list_dimensions=['ga:date' , 'ga:campaign', 'ga:source'],
                   filters = ''):
        """Queries the Analytics Reporting API V4.
        Args:
          analytics: An authorized Analytics Reporting API V4 service object.
          start_date: fecha de inicio del reporte (Ej: 2021-04-14)
          end_date: fecha final del reporte (Ej: 2021-04-14)
          list_metrics: lista con las diferentes métricas
          list_dimensions: lista con las diferentes dimensiones
          filters: string con el filtro (Ej: 'ga:source=~bing' o 'ga:sessions>=600,ga:users>=801')
          DOC filters:   

        Returns:
          response: json con los resultados
        """
        metrics = []
        for i in list_metrics:
            metrics.append({"expression": i})

        dimensions = []
        for i in list_dimensions:
            dimensions.append({"name": i})

        response = self.analytics.reports().batchGet(
            body={
                'reportRequests': [
                    {
                        'viewId': view_id,
                        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
                        'metrics': metrics,
                        'dimensions': dimensions,
                        'filtersExpression': filters,
                        'pageSize' : 100000
                    }]
            }
        ).execute()
        
        return ga_response_dataframe(response)

In [47]:
sys.path.append('C:\\Users\\c_gra\\OneDrive\\Escritorio\\ETL_Datafactory')

In [48]:
from scripts import config

In [49]:
with open('C:/Users/c_gra/OneDrive/Escritorio/ETL_Datafactory/data/cfs/entrypoint_gads.json') as f:
    dict_entrypoint = json.load(f)

In [50]:
dict_entrypoint

{'platform': 'googleads',
 'ecommerce': False,
 'advertiser_name': 'vitaldent_2022',
 'advertiser_id': '189183554',
 'list_dimensions_insights': ['ga:date',
  'ga:campaign',
  'ga:adGroup',
  'ga:adwordsCampaignID',
  'ga:adwordsAdGroupID',
  'ga:adwordsCustomerID'],
 'list_dimensions_clientid': ['ga:date',
  'ga:campaign',
  'ga:source',
  'ga:medium',
  'ga:channelGrouping',
  'ga:dimension3',
  'ga:hostname',
  'ga:landingPagePath',
  'ga:fullReferrer'],
 'table_insights': 'ommax.etl_vitaldent_googleAds_insights',
 'table_insights_client_id': 'ommax.etl_vitaldent_tuclinica_clientid'}

In [51]:
list_metrics = [ 'ga:adCost' , 'ga:impressions' , 'ga:adClicks' , 'ga:sessions',
'ga:goal1Completions', 'ga:goal2Completions', 'ga:goal3Completions', 'ga:goal4Completions',
'ga:goalCompletionsAll']

In [55]:
view_id = dict_entrypoint['advertiser_id']

dateRanges = {'start_date' : '2022-12-01',
              'end_date' : '2023-01-23'}

list_dimensions = dict_entrypoint['list_dimensions_insights']

In [53]:
filters = 'ga:adGroup!=(not set)'

In [56]:
Client.get_report(view_id, datetime['start_date'], datetime['end_date'], list_metrics, list_dimensions, filters)

TypeError: 'type' object is not subscriptable